# Finding the most usful predictor of regions of temperature varibiity

## Set 1: Clustering 

First step is to cluster areas of the world into unique clusters to reduce the dimensionality of our data and save on computation time while capturing the information from within the data. 

## Step 2: Feature selection 

Find the single most useful predictor for each cluster using the `Boruta` package for random forrest feature selection 

In [1]:
library(matrixStats)
library(HiClimR)
library(ncdf4)
library(pdist)
library(Boruta)
library(ggplot2)

## Step 1: CLustering 

First we need to load our data and do some processing before we can preform our clustering. Firsly we load our data, it is in the form of a netcdf so we use the `ncdf4` package. 

In [2]:
tas_jja <- nc_open("../data/netcdf/tas_jja_1981_2016_intersected_shortglobe.nc")

Next we need to extract our data from our netcdf. The `ncvar_get` function does this for us. 

We then use the `grid2D` function to create a grid of latitude and longitudes which we then put into a list so we can use this with the `HiClimR` package. 

The final line creates a matrix in the form that `HiClimR` expects.

In [3]:
nc <- ncvar_get(tas_jja, "monthly_mean_2_metre_temperature")
lon <- ncvar_get(tas_jja, "longitude")
lat <- ncvar_get(tas_jja, "latitude", verbose = F)
t <- ncvar_get(tas_jja, "time")
xGrid <- grid2D(lon = unique(lon), lat = unique(lat))
lonn <- c(xGrid$lon)
latt <- c(xGrid$lat)
n <- aperm(nc, c(3,2,1))
x <- t(matrix(n, nrow=dim(n)[1], byrow=FALSE))

Now we coursen our data as to avoid some memory problems, then we extract the variables again to use in the clustering function

In [4]:
xc <- coarseR(x = x, lon = lonn, lat = latt, lonStep = 6, latStep = 6)
lon <- xc$lon
lat <- xc$lat
x <- xc$x

Now for the fun bit, Clustering! 

The function takes our data `x` as well as our coordinates `lat` and `lon`. We set the number of clusters `k` to be 35. 

I also use wards method of clustering. `detrend` and `standardize` hyperparameters are both set to true to remove any climate change signal and to ensure we cluster based on variability and not the magnitude of the temperature in that cluster. 

`validClimR` allows us to extract our cluster means and look at other useful infomration. We can also set the minimum size of a cluster. It also allows a `minSize` to be selected which drops clusters below a certain size. 

In [5]:
k = 35 
y <- HiClimR(x, lon = lon, lat = lat, lonStep = 1, latStep = 1, geogMask = FALSE, gMask = gmask,
         meanThresh = NULL, varThresh = 0, detrend = TRUE,standardize = TRUE, nPC = NULL,
         method = "ward", hybrid = FALSE, kH = NULL,members=NULL,nSplit = 1,upperTri = TRUE,
         verbose = FALSE,validClimR = TRUE, k = k, minSize = 1, alpha = 0.05,plot = TRUE,
         colPalette = NULL, hang = -1,labels = FALSE)
z <- validClimR(y, k = k, minSize = 5, alpha = 0.05, plot = TRUE, colPalette = NULL)


Running Time:
   user  system elapsed 
185.886  21.245 207.120 
Time difference of 3.452001 mins
---> Computing cluster means...
---> Computing inter-cluster correlations...
---> Computing intra-cluster correlations...
---> Computing summary statistics...
Generating region map...


In [ ]:
y.nc <- HiClimR2nc(y=y, ncfile="../data/netcdf/HiClimR_k35_ward_shortglobe.nc", timeunit="years")

Now we can extract each clusters mean and how many points are in each cluster. `clust_means` is a dataframe 

In [6]:
clust_means <- z$clustMean
clust_sizes <- z$clustSize

Here are the printed cluster sizes 

In [7]:
clust_sizes

cutTree
   1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
 688 1178  590  550 1170  643  729  826  833  300  459  552  775 1740  460  562 
  17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32 
 680 2072  859  301  711  637  468  641  825  569  672  431  376  610  408  522 
  33   34   35 
 364  547  252 

# Step 2: feature selection 

Now we have the data we need to move onto step 2. Each cluster mean will be used as the varible we want to find the most important climate index for predicting this varible. The most important climate index for each cluster will then be stored and saved. 


In [8]:
empty_matrix <- function(rownames, colnames){
    empty_array = array(NA, c(length(rownames), length(colnames)))
    rownames(empty_array) = rownames
    colnames(empty_array) = colnames 
    return(empty_array)
}

Load climate index dataframe

In [9]:
climate_indices_fname = '../data/csv/climate_indices_1981-2016.csv'
climate_indices = read.csv(climate_indices_fname, header = TRUE,sep = ",", stringsAsFactors = FALSE)
colnames(climate_indices) <- gsub("[[:punct:]]", "-", colnames(climate_indices))
head(climate_indices)

,wpsh-9,wpsh-10,wpsh-11,wpsh-12,wpsh-1,wpsh-2,wpsh-3,wpsh-4,wpsh-5,iod-9,⋯,mjopc1b-5,amo-9,amo-10,amo-11,amo-12,amo-1,amo-2,amo-3,amo-4,amo-5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1981,14647.72,14855.22,14874.19,14754.29,14840.38,14823.92,14974.42,14865.94,14733.79,-0.639,⋯,0.3135853,-0.14,-0.15,-0.15,-0.15,-0.15,-0.16,-0.16,-0.16,-0.16
1982,14724.16,14813.61,14881.97,14928.39,14844.86,14816.04,14831.94,14822.94,14803.69,-0.651,⋯,-0.8869489,-0.16,-0.17,-0.17,-0.17,-0.17,-0.17,-0.17,-0.16,-0.16
1983,14697.37,14787.07,15000.44,14983.69,15086.88,14982.15,14953.99,14939.12,14837.90,0.548,⋯,-1.2200562,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16
1984,14788.31,14886.98,14866.77,14894.84,14774.50,14744.34,14836.41,14743.72,14685.91,0.037,⋯,1.0038773,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15
1985,14756.51,14682.57,14898.16,14755.96,14815.74,14657.85,14918.78,14622.10,14663.60,-0.502,⋯,0.9774278,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15,-0.15,-0.16
1986,14790.06,14870.71,14887.35,14840.96,14812.80,14745.91,14785.91,14862.11,14621.19,-0.132,⋯,-0.3579582,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.17,-0.17


In [10]:
ind_list = c("wpsh", "iod", "pdo", "nin3","seu", "nao", "snao", "scand", "ao", "eawr", "peu", "cgt", "pstratu", "mjopc1a", "mjopc1b", "amo")
cmons_num = c(9,10,11,12,1,2,3,4,5)

Detrend and standardise the data 

In [11]:
cindex_anoms = empty_matrix(rownames(climate_indices), colnames(climate_indices))

for (s in 1:dim(climate_indices)[2]) {
    timesteps = 1:dim(climate_indices)[1]
    detrend = lm(climate_indices[,s] ~ timesteps)$residuals
    standardised = scale(detrend)
    cindex_anoms[,s] = standardised
}

Calculation of significant correlation to remove some poorly correlated indices 

In [12]:
critical.r <- function(n, alpha) {
  df <- n - 2
  critical.t <- qt( alpha/2, df, lower.tail = F )
  critical.r <- sqrt( (critical.t^2) / ( (critical.t^2) + df ) )
  return( critical.r )
}

In [13]:
cindex_clust_cors = cor(cindex_anoms, clust_means)
head(cindex_clust_cors)

,1,2,3,4,5,6,7,8,9,10,⋯,26,27,28,29,30,31,32,33,34,35
wpsh-9,0.005799848,0.04057066,-0.03083227,0.07987746,0.34071888,-0.01630493,-0.1327423,-0.07438731,-0.09759056,0.05521589,⋯,-0.04114264,-0.01293670,0.05608885,0.10282675,0.272139075,0.19957481,0.33533929,-0.23153322,0.12703304,-0.153437362
wpsh-10,-0.209234410,0.15943928,-0.08073953,0.15544845,0.29973523,-0.14968606,0.2190246,0.12403121,-0.24438560,0.03497221,⋯,-0.02766371,-0.09195440,0.34467009,0.25308461,0.402487952,0.17423682,0.06418368,-0.12082906,0.15708375,0.006874142
wpsh-11,0.300547045,0.41659399,0.02758540,0.05340974,0.03974924,0.16667733,0.2813868,0.55624825,-0.11489237,-0.11149958,⋯,-0.01577221,-0.05943758,0.01281907,-0.03064276,-0.112357857,-0.19484284,0.29649734,0.12001687,0.05488988,0.084278401
wpsh-12,0.249913571,0.29880010,0.06480233,-0.07449971,-0.17507538,0.20959044,0.3695751,0.34030814,-0.13514397,-0.14893093,⋯,-0.41931316,0.04992801,0.16549021,-0.24402059,-0.052905291,-0.16553310,0.13557951,0.30972555,0.07959073,-0.235481390
wpsh-1,0.309945686,0.43622508,-0.07302310,-0.07352957,-0.26879666,0.12591420,0.2851845,0.46000273,0.02370832,-0.22009252,⋯,-0.14486715,-0.02935098,-0.01416633,-0.11254153,-0.090480149,-0.15617051,0.03730905,0.05234819,0.17230562,0.058013541
wpsh-2,0.317700754,0.55393058,0.03430030,0.07172785,-0.08085522,0.04370607,0.3218175,0.49824815,0.04793476,0.06128107,⋯,-0.02383256,0.13284639,0.15807312,-0.02723239,-0.000834714,-0.04939049,0.11740895,0.32572721,0.08614723,-0.048018540


Calculate the crictal correlation threshold 

In [14]:
cor_crit = critical.r(length(rownames(cindex_anoms)), 0.15)
cor_crit

[1] 0.2448937

The names of all the remaining climate indices 

In [15]:
cindex_crit_names = colnames(cindex_anoms)[unique(which(abs(cindex_clust_cors) >= cor_crit, arr.ind=TRUE)[,1])]
cindex_crit_names

[1] "wpsh-11"    "wpsh-12"    "wpsh-1"     "wpsh-2"     "wpsh-5"    
  [6] "iod-12"     "nin3-10"    "nin3-11"    "nin3-12"    "nin3-1"    
 [11] "nin3-2"     "seu-1"      "seu-2"      "seu-4"      "nao-10"    
 [16] "nao-11"     "nao-1"      "nao-3"      "nao-4"      "snao-2"    
 [21] "snao-4"     "scand-12"   "scand-1"    "ao-9"       "ao-11"     
 [26] "ao-12"      "ao-1"       "ao-3"       "cgt-1"      "cgt-3"     
 [31] "mjopc1a-10" "mjopc1a-11" "mjopc1a-5"  "mjopc1b-2"  "wpsh-3"    
 [36] "wpsh-4"     "iod-9"      "iod-10"     "iod-11"     "iod-1"     
 [41] "pdo-9"      "pdo-10"     "pdo-11"     "pdo-2"      "pdo-3"     
 [46] "nin3-9"     "nin3-3"     "snao-11"    "snao-5"     "ao-4"      
 [51] "eawr-11"    "eawr-12"    "eawr-2"     "eawr-5"     "pstratu-9" 
 [56] "pstratu-10" "pstratu-3"  "pstratu-5"  "mjopc1a-9"  "mjopc1a-2" 
 [61] "mjopc1b-9"  "mjopc1b-10" "mjopc1b-12" "mjopc1b-5"  "amo-4"     
 [66] "amo-5"      "pdo-1"      "pdo-4"      "pdo-5"      "seu-11"    
 [71] "seu-5"      "snao-9"     "scand-9"    "eawr-3"     "eawr-4"    
 [76] "peu-10"     "peu-11"     "pstratu-1"  "pstratu-2"  "mjopc1a-3" 
 [81] "mjopc1b-3"  "nao-5"      "snao-12"    "scand-3"    "peu-1"     
 [86] "peu-5"      "amo-1"      "amo-2"      "amo-3"      "wpsh-9"    
 [91] "wpsh-10"    "iod-3"      "iod-4"      "nin3-4"     "nin3-5"    
 [96] "snao-3"     "scand-11"   "eawr-1"     "peu-2"      "cgt-5"     
[101] "mjopc1a-4"  "mjopc1b-4"  "pdo-12"     "seu-3"      "peu-3"     
[106] "cgt-11"     "cgt-2"      "amo-9"      "amo-10"     "amo-11"    
[111] "amo-12"     "iod-5"      "cgt-9"      "seu-10"     "peu-12"    
[116] "peu-4"      "cgt-10"     "mjopc1b-11" "snao-1"     "scand-10"  
[121] "pstratu-4"  "iod-2"      "nao-9"      "nao-2"      "scand-5"   
[126] "cgt-12"     "pstratu-12" "eawr-9"     "eawr-10"    "ao-5"      
[131] "cgt-4"      "mjopc1a-1"  "mjopc1b-1"  "ao-2"       "seu-9"     
[136] "scand-4"    "peu-9"      "pstratu-11" "ao-10"      "nao-12"    
[141] "seu-12"     "scand-2"    "mjopc1a-12"

### Feature selection using random forests

We use a random forest regression model from the Boruta package to robustly rank predictors in terms of their explanatory power with regards to the variance in regional temperatures.

Random forests rank the explanatory power of features by their “importance”. Typically, there is no measure of statistical significance for feature importance, resulting in ambiguity about which values are robustly different from zero. Boruta implements a different approach, creating randomised “shadow” versions of each feature and using these as additional input features. After 1000 trials, we select features that give out-of-sample predictions which statistically outperform the best shadow feature at the 5% significance level.

Here we use this to select the feature that has the maximum importance

In [16]:
x_train <- cindex_anoms[, cindex_crit_names]
pcrit = 0.05

In [17]:
do_boruta_stuff_get_most_important <- function(df_in){
    df_in  = data.frame(cbind(x_train, y_train)) 
    boruta_output <- Boruta(y_train ~ ., data=df_in, doTrace=0, maxRuns= 400, ntree=4000, pValue = 0.1)
    confirmed_df <- attStats(boruta_output)[which(attStats(boruta_output)$decision == "Confirmed") ,]
    return(rownames(confirmed_df[which.max(confirmed_df$meanImp),]))
    }

In [18]:
empty_array = array(NA, c(length(colnames(clust_means)), 1))

Find the most important feature for each clustered region 

In [19]:
for (k in 1:dim(clust_means)[2]) {
    set.seed(1)
    y_train <- clust_means[ ,k]
    df_in  = data.frame(cbind(x_train, y_train)) 
    empty_array[k,] = do_boruta_stuff_get_most_important(df_in)
    }

Selected predictors 

In [20]:
empty_array

nin3.1
mjopc1a.5
pdo.5
peu.1
nin3.5
pdo.5
nin3.5
nin3.12
nin3.5
peu.4
seu.3


Save selected predictors 

In [21]:
write.csv(empty_array, "../data/csv/predictors_for_clusters_k35_ward.csv", row.names=FALSE)
